In [1]:
import sys
sys.path.append('..')
from utils import *
sys.path.append('../attacks/')
from fgsm import *
from pgd import *
sys.path.append('../models/')
import torch
from torchvision import transforms
import numpy as np
from torchvision import datasets
from torch import norm
from tqdm import tqdm
import cv2
from torch import nn

In [2]:
imgs = datasets.ImageFolder('~/dataset/imagenet1000/val')

In [3]:
wm = cv2.imread('../img/logo.jpg')
wm = cv2.cvtColor(wm,cv2.COLOR_BGR2RGB)

In [4]:
def attack_and_record(filename: str,model: nn.Module, imgs:datasets, wm: np.ndarray):
    block_size = 8
    alpha = 0.1
    beta = 1/255
    l1 = 0.001
    l2 = 0.001
    s_a = 0.000005
    s_b = 0.00001
    N = 10
    with open(filename,'a') as file:
        for i in tqdm(range(len(imgs)), desc='Processing'):
            img = np.array(imgs[i][0])
            img = addborder(img,block_size)
            img = transforms.ToTensor()(img).cuda()
            target = torch.tensor([imgs[i][1]]).cuda()
            pred_label = model(img.unsqueeze(0)).argmax().item()
            wm_to_embed = cv2.resize(wm,dsize=(img.shape[2],img.shape[1]),fx=1,fy=1,interpolation=cv2.INTER_LINEAR)
            wm_to_embed = transforms.ToTensor()(wm_to_embed).cuda()
            perd_img,wm_extracted = fgsm_wm_opti(img,target,wm_to_embed,model,alpha,beta,block_size,N,l1,l2,s_a,s_b)
            res = model(perd_img.unsqueeze(0))
            perd_label = res.argmax().item()
            l2_norm = norm(wm_extracted-wm_to_embed).item()
            file.write('{},{},{},{},{}\n'.format(i,imgs[i][1],pred_label,perd_label,l2_norm))


In [7]:
from vgg_19 import *
vgg_in = vgg19_IN()
vgg_in = vgg_in.cuda()
vgg_in = vgg_in.eval()
filename = '/home/hancy/code/adv_wm/record/220927_fgsm_wm_vgg_19.txt'
attack_and_record(filename,vgg_in,imgs,wm)

Processing: 100%|██████████| 3923/3923 [50:20<00:00,  1.30it/s]  
